In [2]:
import PIL
from torchmetrics.image import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
from torchvision import transforms

def get_test_img(file):
    test_img = PIL.Image.open(f'test pictures\\{file}')
    return transforms.ToTensor()(test_img)

In [4]:
import PIL
import numpy as np
import torch
import torchvision
from torchvision.transforms.v2 import Compose, GaussianBlur, RandomEqualize, RandomSolarize, RandomApply
import wandb
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from Dataset.AerialDataset import AerialDataset
from tasks.SRDiffTrainer import SRDiffTrainer
from models.SRDIFFBuilder import SRDiffBuilder
from utils.model_utils import load_model

#Data
lr_size = 64
hr_size = 256
batch_size = 20
dataset_dir = 'E:\\TFG\\dataset_tfg'

transforms = Compose(
    [RandomApply(transforms= [GaussianBlur(7)], p = 0.5),
    RandomEqualize()]
)

dataset = AerialDataset(dataset_dir, lr_size, hr_size, data_augmentation = None, aux_sat_prob= 0.4, sat_dataset_path= "E:\\TFG\\dataset_tfg\\satelite_dataset\\64_256")
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6, 0.2, 0.2], generator=torch.Generator().manual_seed(420))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
device = torch.device(0)

In [6]:
from tqdm import tqdm
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionUpscalePipeline
import torch
device = torch.device(0)

# load model and scheduler
model_id = "stabilityai/stable-diffusion-x4-upscaler"
pipeline = StableDiffusionUpscalePipeline.from_pretrained(
    model_id, revision="fp16", torch_dtype=torch.float16
)
wandb.login()
hyperparams = {
    "pretrained": True,
    "fine tunning": False,
    "batch_size": batch_size
}
wandb.init(project="Stable diffusion", config=hyperparams, name=model_id + "no tuning")
pipeline = pipeline.to(device)
metrics = {"psnr":0, "ssim":0}
for batch in tqdm(test_dataloader):
    bicubic = batch["bicubic"]
    hr = batch["hr"]
    prompt = ["Satelite imagery"]*batch_size
    sr = pipeline(prompt=prompt, image=bicubic).images[0]
    ssim = StructuralSimilarityIndexMeasure().to(device=device)
    psnr = PeakSignalNoiseRatio().to(device=device)
    metrics['psnr'] += psnr(sr, hr)
    metrics['ssim'] += ssim(sr, hr)

metrics = {metric: value / len(test_dataloader) for metric, value in metrics.keys()}
wandb.log(metrics)
wandb.finish()

C:\Users\adria\miniconda3\envs\cuda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
vae\diffusion_pytorch_model.safetensors not found
C:\Users\adria\miniconda3\envs\cuda\Lib\site-packages\diffusers\pipelines\stable_diffusion\pipeline_stable_diffusion_upscale.py:125: FutureWarning: The configuration file of the vae does not contain `scaling_factor` or it is set to 0.18215, which seems highly unlikely. If your checkpoint is a fine-tuned version of `stabilityai/stable-diffusion-x4-upscaler` you should change 'scaling_factor' to 0.08333 Please make sure to update the config accordingly, as not doing so might lead to incorrect results in future versions. If you have downloaded this checkpoint from the Hugging Face Hub, it would be very nice if you could open a Pull Request for the

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

C:\Users\adria\miniconda3\envs\cuda\Lib\site-packages\diffusers\models\attention_processor.py:1129: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  hidden_states = F.scaled_dot_product_attention(
  0%|          | 0/40 [01:08<?, ?it/s]


KeyboardInterrupt: 